In [147]:
%load_ext autoreload
%autoreload 2
import collections
import pandas as pd
import numpy as np
from screening.validation.crossval import crossval_table, crossval_ref_filter, crossval_max_value_filter
from pprint import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [167]:

def create_op_dict( op_name, extra_suf="" ):

    d = collections.OrderedDict( {
                'max_sp'           : f'summary{extra_suf}/max_sp',
                'auc'              : f'summary{extra_suf}/auc',
                #'acc'              : f'summary{extra_suf}/acc',
                #'pd'               : f'summary{extra_suf}/pd',
                #'fa'               : f'summary{extra_suf}/fa',
                'sens'             : f'summary{extra_suf}/sensitivity',
                'spec'             : f'summary{extra_suf}/specificity',
                'threshold'        : f'summary{extra_suf}/threshold',
                'roc'              : f'summary{extra_suf}/roc',
                'roc_val'          : f'summary{extra_suf}/roc_val',
                'roc_op'           : f'summary{extra_suf}/roc_op',
                'roc_test'         : f'summary{extra_suf}/roc_test',

                'min_spec_sens_reached' : f'{op_name}{extra_suf}/min_spec_sens_reached',

                'max_sp_val'       : f'summary{extra_suf}/max_sp_val',
                'auc_val'          : f'summary{extra_suf}/auc_val',
                #'acc_val'          : f'summary{extra_suf}/acc_val',
                #'pd_val'           : f'summary{extra_suf}/pd_val',
                #'fa_val'           : f'summary{extra_suf}/fa_val',
                'sens_val'         : f'summary{extra_suf}/sensitivity_val',
                'spec_val'         : f'summary{extra_suf}/specificity_val', 

                'max_sp_test'      : f'summary{extra_suf}/max_sp_test',
                'auc_test'         : f'summary{extra_suf}/auc_test',
                #'acc_test'         : f'summary{extra_suf}/acc_test',
                #'pd_test'          : f'summary{extra_suf}/pd_test',
                #'fa_test'          : f'summary{extra_suf}/fa_test',
                'sens_test'        : f'summary{extra_suf}/sensitivity_test',
                'spec_test'        : f'summary{extra_suf}/specificity_test', 

                'max_sp_op'        : f'summary{extra_suf}/max_sp_op',
                'auc_op'           : f'summary{extra_suf}/auc_op',
                #'acc_op'           : f'summary{extra_suf}/acc_op',
                #'pd_op'            : f'summary{extra_suf}/pd_op',
                #'fa_op'            : f'summary{extra_suf}/fa_op',
                'sens_op'          : f'summary{extra_suf}/sensitivity_op',
                'spec_op'          : f'summary{extra_suf}/specificity_op', 

                'sp_index'         : f'{op_name}{extra_suf}/sp_index',
                'sens_at'          : f'{op_name}{extra_suf}/sensitivity',
                'spec_at'          : f'{op_name}{extra_suf}/specificity',
                'acc_at'           : f'{op_name}{extra_suf}/acc',
                'threshold_at'     : f'{op_name}{extra_suf}/threshold',

                'sp_index_val'     : f'{op_name}{extra_suf}/sp_index_val',
                'sens_at_val'      : f'{op_name}{extra_suf}/sensitivity_val',
                'spec_at_val'      : f'{op_name}{extra_suf}/specificity_val',
                #'acc_at_val'       : f'{op_name}{extra_suf}/acc_val',
                'threshold_at_val' : f'{op_name}{extra_suf}/threshold_val',

                'sp_index_test'    : f'{op_name}{extra_suf}/sp_index_test',
                'sens_at_test'     : f'{op_name}{extra_suf}/sensitivity_test',
                'spec_at_test'     : f'{op_name}{extra_suf}/specificity_test',
                #'acc_at_test'      : f'{op_name}{extra_suf}/acc_test',
                'threshold_at_test': f'{op_name}{extra_suf}/threshold_test',

                'sp_index_op'      : f'{op_name}{extra_suf}/sp_index_op',
                'sens_at_op'       : f'{op_name}{extra_suf}/sensitivity_op',
                'spec_at_op'       : f'{op_name}{extra_suf}/specificity_op',
                #'acc_at_op'        : f'{op_name}{extra_suf}/acc_op',
                'threshold_at_op'  : f'{op_name}{extra_suf}/threshold_op',

                'inference'        : f'{op_name}{extra_suf}/inference',
        })
    return d


#extra_suf='_val'
extra_suf=''
conf_dict = collections.OrderedDict(
    {
        'sens90'   : create_op_dict( 'sens90'  , extra_suf=extra_suf ),
        'max_sp'   : create_op_dict( 'max_sp' , extra_suf=extra_suf),
        'spec70'   : create_op_dict( 'spec70'  , extra_suf=extra_suf),
    }
)
pprint(conf_dict)

OrderedDict([('sens90',
              OrderedDict([('max_sp', 'summary/max_sp'),
                           ('auc', 'summary/auc'),
                           ('sens', 'summary/sensitivity'),
                           ('spec', 'summary/specificity'),
                           ('threshold', 'summary/threshold'),
                           ('roc', 'summary/roc'),
                           ('roc_val', 'summary/roc_val'),
                           ('roc_op', 'summary/roc_op'),
                           ('roc_test', 'summary/roc_test'),
                           ('min_spec_sens_reached',
                            'sens90/min_spec_sens_reached'),
                           ('max_sp_val', 'summary/max_sp_val'),
                           ('auc_val', 'summary/auc_val'),
                           ('sens_val', 'summary/sensitivity_val'),
                           ('spec_val', 'summary/specificity_val'),
                           ('max_sp_test', 'summary/max_sp_test'),
                

In [168]:
models = [
    ( 'user.philipp.gaspar.convnets_v1.baseline.shenzhen_santacasa.exp.20240303.r1'                          , 'base.sh-sc.e'        ),
    #( 'user.philipp.gaspar.convnets_v1.interleaved.shenzhen_santacasa.exp_wgan_p2p.20240303.r1'              , 'inte.sh-sc.ewp'      ),
]

models_plus_manaus = [
    ( 'user.philipp.gaspar.convnets_v1.baseline.shenzhen_santacasa_manaus.exp.20240303.r1'                          , 'base.sh-sc.e'        ),
    ( 'user.philipp.gaspar.convnets_v1.interleaved.shenzhen_santacasa_manaus.exp_wgan_p2p.20240303.r1'              , 'inte.sh-sc.ewp'      ),
]


In [169]:

basepath='/mnt/brics_data/models/v1'
cv = crossval_table( conf_dict )
for path, train_tag in models:
    cv.fill( basepath+'/'+path , train_tag )
   


2024-03-20 10:01:59.873 | INFO     | screening.validation.crossval:fill:139 - Reading file for base.sh-sc.e tag from /mnt/brics_data/models/v1/user.philipp.gaspar.convnets_v1.baseline.shenzhen_santacasa.exp.20240303.r1
2024-03-20 10:01:59.873 | INFO     | screening.validation.crossval:fill:151 - There are 90 files for this task...
2024-03-20 10:01:59.873 | INFO     | screening.validation.crossval:fill:152 - Filling the table... 
Reading base.sh-sc.e...: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:03<00:00, 27.85it/s]
2024-03-20 10:02:03.109 | INFO     | screening.validation.crossval:fill:190 - End of fill step, a pandas DataFrame was created...


In [170]:
table = cv.table

In [171]:
filters = {
    'sens90' : crossval_ref_filter(0.9, 'sens_op', 'spec_op', test_key='spec_op'),
    'max_sp' : crossval_max_value_filter(sort_key='max_sp_op', test_key='max_sp_op'),
    'spec70' : crossval_ref_filter(0.7, 'spec_op', 'sens_op', test_key='sens_op'),
}

def apply_filters( table , filters):
    best_sorts_list = []
    best_tests_list = []
    for op_name in table.op_name.unique():
        best_sorts = filters[op_name].filter_sorts( table.loc[table.op_name==op_name])
        best_tests = filters[op_name].filter_tests( best_sorts )
        best_sorts_list.append(best_sorts)
        best_tests_list.append(best_tests)
    return pd.concat(best_sorts_list, axis='rows'), pd.concat(best_tests_list, axis='rows')

best_sorts, best_tests = apply_filters(cv.table, filters)

/home/joao.pinto/git_repos/screening/screening/validation/crossval.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['is_in'] = table.apply(lambda row : is_in(row,delta), axis='columns')


KeyError: 'max_sp'

In [172]:
best_tests[['op_name','test','sort','sens_val','spec_val','sens_op','spec_op','sens_test','spec_test']]

,op_name,test,sort,sens_val,spec_val,sens_op,spec_op,sens_test,spec_test
252,sens90,6,8,0.941176,0.895833,0.913907,0.952703,0.735294,0.918367
226,maxsp,1,4,0.818182,0.900000,0.973510,0.986456,0.705882,0.820000
35,spec70,3,1,0.941176,0.680000,0.933993,0.805869,0.757576,0.800000


In [173]:
best_sorts.loc[best_sorts.op_name=='sens90'][['op_name','test','sort','sens_val','spec_val','sens_op','spec_op','sens_test','spec_test']]

,op_name,test,sort,sens_val,spec_val,sens_op,spec_op,sens_test,spec_test
189,sens90,0,0,0.823529,0.860000,0.887417,0.930023,0.852941,0.960000
210,sens90,1,1,0.696970,0.920000,0.910596,0.932280,0.852941,0.860000
21,sens90,2,2,0.757576,0.880000,0.900990,0.918736,0.757576,0.900000
12,sens90,3,8,0.911765,0.833333,0.917492,0.936795,0.757576,0.900000
3,sens90,4,0,0.852941,0.920000,0.900990,0.930023,0.818182,0.860000
120,sens90,5,8,0.911765,0.812500,0.894389,0.900677,0.787879,0.800000
252,sens90,6,8,0.941176,0.895833,0.913907,0.952703,0.735294,0.918367
15,sens90,7,0,0.823529,0.960000,0.894040,0.932584,0.941176,0.895833
42,sens90,8,2,0.636364,0.940000,0.897351,0.937079,0.852941,0.937500
267,sens90,9,2,0.666667,0.900000,0.897351,0.930337,0.911765,0.750000


In [174]:
from pprint import pprint
from tqdm import tqdm
from loguru import logger
from expand_folders import expand_folders
import pickle

basepath='/mnt/brics_data/models/v1'
models = [
    #( 'user.philipp.gaspar.convnets_v1.baseline.shenzhen_santacasa.exp.20240303.r1'                          , 'base.sh-sc.e'        ),
    ( 'user.philipp.gaspar.convnets_v1.interleaved.shenzhen_santacasa.exp_wgan_p2p.20240303.r1'              , 'inte.sh-sc.ewp'      ),
    ( 'user.philipp.gaspar.convnets_v1.baseline.shenzhen_santacasa_manaus.exp.20240303.r1'                          , 'base.sh-sc.e'        ),
    ( 'user.philipp.gaspar.convnets_v1.interleaved.shenzhen_santacasa_manaus.exp_wgan_p2p.20240303.r1'              , 'inte.sh-sc.ewp'      ),
]

for m in models:
    for path in tqdm(expand_folders(basepath+'/'+m[0], filters=['output.pkl'])):
        with open(path, 'rb') as f:
            d = pickle.load(f)
            h = d['history']
            h['sens90'] = h.pop('loose')
            h['max_sp'] = h.pop('medium')
            h['spec70'] = h.pop('tight')

        with open(path, 'wb') as f:
            pickle.dump(d, f)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:59<00:00,  1.50it/s]
